**PART 5: Fine Tuning Model Gemma 3b**

This notebook tries to conduct an experiment into the process of fine-tuning a pre-trained language model. We take a base Gemma3 model (with 12b parameters) and further train it on our specific dataset of political tweets. The objective is to see if we can improve the model's performance and efficiency on our classification task by adapting it to the nuances of our data. However, we haven't achieved convincing results; the code remains to be advanced and modified in the future. The goal was to implement a low training process instead of a real fine-tuning process, which would require much more computational capacity. The low-rank tuning of models with libraries like LORA will introduce a new training process to the original model.

**Structure:**

Imports libraries: including datasets, ollama, and pandas.

Data: Loads both the GPT-4 and Gemma-labeled datasets.

A sample of 100 tweets is selected for this experiment. The data is formatted into a list of dictionaries, each containing a prompt (the tweet text) and a response (the corresponding JSON-formatted label).

Few-Shot Prompt: A  list of few_shot_examples is created with examples of tweet classifications that cover a range of polarization levels and styles. A build_few_shot_prompt function is defined to  construct the prompt. This function combines a detailed set of instructions, the curated few-shot examples, and the new tweet to be classified.

Classification with Few-Shot:

The classify_with_ollama_gemma function sends the constructed few-shot prompt to the local Gemma model. The notebook iterates through the 100-tweet sample, using this advanced prompting technique to generate classifications.

The results are saved to a new CSV file for comparison and analysis.

In [1]:
pip install transformers datasets peft accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 44.6 MB/s eta 0:00:0000:010:01
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6/6 [peft]5/6 [peft]ets]tes]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
s3fs 2025.3.2 requires fsspec==2025.3.2.*, but you have fsspec 2025.3.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install huggingface_hub
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 13.6 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  DEPRECATION: Building 'sentencepiece' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'sentencepiece'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [98 lines of output]
      /opt/anaconda3/lib/python3.13/site-packages/setuptools/_distutils/dist.py:268: UserWarning: Unknown distribution option: 'test_suite'
        warnings.warn(msg)
      running bdist_wheel
      running build
      running build_py
      creating build
 

In [5]:
conda install -c conda-forge sentencepiece

Jupyter detected...
2 channel Terms of Service accepted
doneieving notices: - 
Channels:
 - conda-forge
 - defaults
Platform: osx-arm64
doneecting package metadata (repodata.json): - 
doneing environment: \ 

## Package Plan ##

  environment location: /opt/anaconda3

  added / updated specs:
    - sentencepiece


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2025.8.3   |       hbd8a1cb_0         151 KB  conda-forge
    certifi-2025.8.3           |     pyhd8ed1ab_0         155 KB  conda-forge
    libsentencepiece-0.2.0     |       hc1ab07e_4         880 KB
    sentencepiece-0.2.0        |       hca03da5_4          17 KB
    sentencepiece-python-0.2.0 |  py313h48ca7d4_4         3.1 MB
    sentencepiece-spm-0.2.0    |       h48ca7d4_4          84 KB
    ------------------------------------------------------------
                                           Total:         4

In [32]:
import pandas as pd
from datasets import Dataset

# Load both files
df_gpt = pd.read_csv("data/sample/debate_tweets_sample_1000_for_annotation_labeled.csv")
df_ollama = pd.read_csv("data/sample/debate_tweets_sample_1000_for_annotation_labeled_ollama.csv")

# Restrict both to first 100 rows (where both models overlap)
df_gpt_100 = df_gpt.iloc[:100].copy()
df_ollama_100 = df_ollama.iloc[:100].copy()

In [33]:
data = []

for _, row in df_gpt_100.iterrows():
    prompt = f'Classify the following tweet: "{row["santext"]}"'
    response = {
        "polar_score": row["polar_score"],
        "pole_label": row["pole_label"],
        "valid": row["valid"],
        "comments": row["comments"] if isinstance(row["comments"], str) else ""
    }

    # Add the formatted prompt-response pair to data
    data.append({
        "prompt": prompt,
        "response": str(response)
    })

In [34]:
from datasets import Dataset

dataset = Dataset.from_list(data)

# check first row
print(dataset[0])

{'prompt': 'Classify the following tweet: "rt ryanshead donald trump is dumb as fuuuuuuuuck"', 'response': "{'polar_score': '-5', 'pole_label': 'left', 'valid': 1, 'comments': 'Uses derogatory language towards a right-wing political figure.'}"}


In [35]:
import json

clean_data = []

for _, row in df_gpt_100.iterrows():
    prompt = f'Classify the following tweet: "{row["santext"]}"'
    
    # Create response dictionary
    response_dict = {
        "polar_score": int(row["polar_score"]) if row["polar_score"] != "INV" else "INV",
        "pole_label": row["pole_label"],
        "valid": int(row["valid"]),
        "comments": row["comments"] if isinstance(row["comments"], str) else ""
    }

    # Convert to JSON string
    response_json = json.dumps(response_dict, ensure_ascii=False)

    clean_data.append({
        "prompt": prompt,
        "response": response_json
    })

# Convert to Hugging Face Dataset
from datasets import Dataset
dataset = Dataset.from_list(clean_data)

# check first row
print(dataset[0])

{'prompt': 'Classify the following tweet: "rt ryanshead donald trump is dumb as fuuuuuuuuck"', 'response': '{"polar_score": -5, "pole_label": "left", "valid": 1, "comments": "Uses derogatory language towards a right-wing political figure."}'}


In [36]:
!pip install transformers peft accelerate bitsandbytes

In [48]:
def build_few_shot_prompt(examples, new_tweet):
    base = """
You are a political communication expert analyzing U.S. political tweets. For each tweet, classify polarization using the following:

- polar_score: from -7 (extreme anti-right) to +7 (extreme anti-left). Use ideological tone, sarcasm, name-calling, and issue positioning.
- pole_label: 'left', 'right', or 'neutral'
- valid: 1 if tweet refers to U.S. debate, political figures (e.g., Trump, Biden, Harris, MAGA), 0 if off-topic
- comments: short justification (optional)

Rules (simplified from expert logic):
- Score -7 to -5: Strongly critical of Trump/right (e.g., “Trump is a fascist”, “felon”, “MAGA cult”)
- Score -4 to -2: Pro-Biden/Kamala or mocking right (“Kamala wins”, “vote Biden”, “Trump unhinged”)
- Score -1: Mild or sarcastic criticism of right
- Score 0: Neutral or descriptive (“debate night”, “town hall”)
- Score +1 to +3: Pro-Trump/MAGA, anti-Biden or Kamala (“Trump won”, “vote Trump”, “Kamala clueless”)
- Score +4 to +7: Harsh attacks on Biden/Harris (“pedo Joe”, “communist”, “bloodbath”)

Output strictly in JSON format:
{ "polar_score": -4, "pole_label": "left", "valid": 1, "comments": "Harsh criticism of Trump using ideological slurs." }

Now classify:
"""
    for ex in examples:
        # Extract tweet from the prompt text for clarity
        match = re.search(r'"(.*?)"', ex["prompt"])
        tweet_text = match.group(1) if match else "UNKNOWN TWEET"
        base += f'Tweet: "{tweet_text}"\nResponse: {ex["response"]}\n\n'

    base += f'Classify this new tweet:\nTweet: "{new_tweet}"\nResponse:'
    return base

In [39]:
few_shot_examples = [
    {
        "prompt": 'Classify the following tweet: "Trump is a fascist clown destroying America."',
        "response": '{ "polar_score": -6, "pole_label": "left", "valid": 1, "comments": "Direct attack using fascist framing." }'
    },
    {
        "prompt": 'Classify the following tweet: "Kamala destroyed Trump. President Harris 2024!"',
        "response": '{ "polar_score": -5, "pole_label": "left", "valid": 1, "comments": "Strongly pro-Kamala, mocking Trump." }'
    },
    {
        "prompt": 'Classify the following tweet: "Breaking: The debate is starting now on ABC."',
        "response": '{ "polar_score": 0, "pole_label": "neutral", "valid": 1, "comments": "Factual and neutral." }'
    },
    {
        "prompt": 'Classify the following tweet: "Biden is a senile puppet. Bring back Trump!"',
        "response": '{ "polar_score": 6, "pole_label": "right", "valid": 1, "comments": "Extreme insult toward Biden, strong partisan tone." }'
    },
    {
        "prompt": 'Classify the following tweet: "RT if you think Kamala is clueless."',
        "response": '{ "polar_score": 3, "pole_label": "right", "valid": 1, "comments": "Criticism of Kamala, typical MAGA framing." }'
    },
    {
        "prompt": 'Classify the following tweet: "Kamala is a communist puppet!"',
        "response": '{ "polar_score": 6, "pole_label": "right", "valid": 1, "comments": "Extreme partisan insult using Cold War language." }'
    },
    {
        "prompt": 'Classify the following tweet: "Biden is better than Trump. That’s it."',
        "response": '{ "polar_score": -3, "pole_label": "left", "valid": 1, "comments": "Mild support for Biden." }'
    },
    {
        "prompt": 'Classify the following tweet: "Watching the debate and eating popcorn 🍿"',
        "response": '{ "polar_score": 0, "pole_label": "neutral", "valid": 1, "comments": "No ideological tone, just observation." }'
    },
    {
        "prompt": 'Classify the following tweet: "I miss Obama. These guys are clowns."',
        "response": '{ "polar_score": -1, "pole_label": "left", "valid": 1, "comments": "Mild criticism of current figures, vague pro-Democrat nostalgia." }'
    },
    {
        "prompt": 'Classify the following tweet: "Crypto is the future. #Bitcoin"',
        "response": '{ "polar_score": "INV", "pole_label": "inv", "valid": 0, "comments": "Not related to political debate." }'
    }
]

In [49]:
import ollama
import json
import re

def classify_with_ollama_gemma(tweet, examples):
    prompt = build_few_shot_prompt(examples, tweet)
    response = ollama.chat(model='gemma3:4b', messages=[
        {'role': 'user', 'content': prompt}
    ])

    content = response['message']['content']
    
    # Optional debug print
    print("🔍 Raw model output:\n", content)

    # Remove Markdown code fences (e.g., ```json\n...\n```)
    content_cleaned = re.sub(r"^```(?:json)?\s*|\s*```$", "", content.strip(), flags=re.IGNORECASE | re.MULTILINE)

    try:
        return json.loads(content_cleaned)
    except Exception as e:
        return {
            "polar_score": "ERR",
            "pole_label": "err",
            "valid": 0,
            "comments": f"Parse error: {e}\nRaw response: {content}"
        }

In [50]:
import pandas as pd
from tqdm import tqdm

df = pd.read_csv("data/sample/debate_tweets_sample_1000_for_annotation.csv").head(100)

results = []
for tweet in tqdm(df['santext'], desc="Classifying with Few-Shot Prompting"):
    # The issue is likely in the classify_with_ollama_gemma function
    # It's trying to access tweet['tweet'] instead of using tweet directly
    # Modified to pass tweet as the text content directly
    result = classify_with_ollama_gemma(tweet, few_shot_examples)
    results.append(result)

# Add columns
df['polar_score'] = [r['polar_score'] for r in results]
df['pole_label'] = [r['pole_label'] for r in results]
df['valid'] = [r['valid'] for r in results]
df['comments'] = [r['comments'] for r in results]

df.to_csv("data/sample/debate_tweets_sample_1000_fewshot_ollama.csv", index=False)

Classifying with Few-Shot Prompting:   1%|      | 1/100 [00:12<21:15, 12.88s/it]

🔍 Raw model output:
 ```json
{
  "polar_score": 7,
  "pole_label": "left",
  "valid": 1,
  "comments": "Extremely harsh and abusive language directed at Trump, clearly partisan and aggressively anti-right."
}
```


Classifying with Few-Shot Prompting:   2%|      | 2/100 [00:17<13:32,  8.29s/it]

🔍 Raw model output:
 ```json
{
  "polar_score": -5,
  "pole_label": "left",
  "valid": 1,
  "comments": "Sarcastic and critical of Trump, referencing a fabricated claim to portray him as delusional and dangerous."
}
```


Classifying with Few-Shot Prompting:   3%|▏     | 3/100 [00:23<11:05,  6.86s/it]

...

🔍 Raw model output:
 ```json
{
  "polar_score": 4,
  "pole_label": "right",
  "valid": 1,
  "comments": "Strongly supportive of Trump, praising his debate performance and using patriotic framing."
}
```


Classifying with Few-Shot Prompting:  99%|████▉| 99/100 [09:11<00:05,  5.63s/it]

🔍 Raw model output:
 ```json
{
  "polar_score": 4,
  "pole_label": "right",
  "valid": 1,
  "comments": "Strongly critical of Kamala Harris, using language suggesting she is dishonest and demanding a response.  Partisan framing targeting a specific news personality (Eric Bolling) and referencing a key political figure (Kamala Harris)."
}
```


Classifying with Few-Shot Prompting: 100%|████| 100/100 [09:17<00:00,  5.57s/it]

🔍 Raw model output:
 ```json
{
  "polar_score": 6,
  "pole_label": "right",
  "valid": 1,
  "comments": "Strongly pro-Trump, referencing specific claims and aggressively criticizing Biden/Harris. Uses inflammatory language ('pedo Joe')."
}
```


Now, we test the classification made by Gemma3:4b

In [23]:
import ollama
import json
import pandas as pd

# Few-shot examples
few_shot_examples = [
    {
        "prompt": 'Classify the following tweet: "Biden is a neoliberal fraud."',
        "response": '{ "polar_score": -4, "pole_label": "left", "valid": 1, "comments": "" }'
    },
    {
        "prompt": 'Classify the following tweet: "Trump just destroyed the debate!"',
        "response": '{ "polar_score": 5, "pole_label": "right", "valid": 1, "comments": "" }'
    }
]

In [24]:
def build_few_shot_prompt(examples, tweet):
    prompt = ""
    for ex in examples:
        prompt += f"{ex['prompt']}\nResponse: {ex['response']}\n\n"
    prompt += f'Classify the following tweet: "{tweet}"\nResponse:'
    return prompt

In [25]:
def classify_with_ollama_gemma(tweet, examples):
    prompt = build_few_shot_prompt(examples, tweet)
    response = ollama.chat(model='gemma3:4b', messages=[
        {'role': 'user', 'content': prompt}
    ])
    try:
        return json.loads(response['message']['content'])
    except Exception as e:
        return {
            "polar_score": "ERR",
            "pole_label": "err",
            "valid": 0,
            "comments": f"Parse error: {e}"
        }

In [26]:
df = pd.read_csv("data/sample/debate_tweets_sample_1000_for_annotation.csv")
sample_df = df.head(5)  # Use the first 5 tweets for testing

In [27]:
results = []
for tweet in sample_df['santext']:
    result = classify_with_ollama_gemma(tweet, few_shot_examples)
    results.append(result)

# Add new columns to the dataframe
sample_df['polar_score'] = [r["polar_score"] for r in results]
sample_df['pole_label'] = [r["pole_label"] for r in results]
sample_df['valid'] = [r["valid"] for r in results]
sample_df['comments'] = [r["comments"] for r in results]

#  results
sample_df[['santext', 'polar_score', 'pole_label', 'valid', 'comments']]

/var/folders/8x/qy8j1cyj36jd9vpnlk90r0hh0000gn/T/ipykernel_72338/1575122912.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_df['polar_score'] = [r["polar_score"] for r in results]
/var/folders/8x/qy8j1cyj36jd9vpnlk90r0hh0000gn/T/ipykernel_72338/1575122912.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_df['pole_label'] = [r["pole_label"] for r in results]
/var/folders/8x/qy8j1cyj36jd9vpnlk90r0hh0000gn/T/ipykernel_72338/1575122912.py:9: SettingWithCopyWarning: 
A value is trying to be s

,santext,polar_score,pole_label,valid,comments
0,rt ryanshead donald trump is dumb as fuuuuuuuuck,ERR,err,0,Parse error: Expecting value: line 1 column 1 ...
1,rt politico woof why is trump claiming immigra...,ERR,err,0,Parse error: Expecting value: line 1 column 1 ...
2,nevermind rt robbysoave trump my retribution i...,ERR,err,0,Parse error: Expecting value: line 1 column 1 ...
3,rt leadingreport breaking former president tru...,ERR,err,0,Parse error: Expecting value: line 1 column 1 ...
4,rt projectlincoln abc jd vance says you would ...,ERR,err,0,Parse error: Expecting value: line 1 column 1 ...


In [28]:
def classify_with_ollama_gemma(tweet, examples):
    prompt = build_few_shot_prompt(examples, tweet)
    response = ollama.chat(model='gemma3:4b', messages=[
        {'role': 'user', 'content': prompt}
    ])

    content = response['message']['content']
    print("Raw model output:\n", content) 

    try:
        return json.loads(content)
    except Exception as e:
        return {
            "polar_score": "ERR",
            "pole_label": "err",
            "valid": 0,
            "comments": f"Parse error: {e}\nRaw response: {content}"
        }

In [29]:
tweet = sample_df.iloc[0]['santext']
result = classify_with_ollama_gemma(tweet, few_shot_examples)
print(result)

🔍 Raw model output:
 ```json
{
  "polar_score": 7,
  "pole_label": "right",
  "valid": 1,
  "comments": "This tweet contains strong, abusive language and is highly polarized due to its personal attack and aggressive sentiment. The use of 'fuuuuuuuuck' significantly elevates the negativity."
}
```
{'polar_score': 'ERR', 'pole_label': 'err', 'valid': 0, 'comments': 'Parse error: Expecting value: line 1 column 1 (char 0)\nRaw response: ```json\n{\n  "polar_score": 7,\n  "pole_label": "right",\n  "valid": 1,\n  "comments": "This tweet contains strong, abusive language and is highly polarized due to its personal attack and aggressive sentiment. The use of \'fuuuuuuuuck\' significantly elevates the negativity."\n}\n```'}
